In [1]:
%cd ..

/home/dimits/Documents/research/llm_moderation/experiments


In [2]:
from tasks import preprocessing


CONVERSATION_DIR = "data/generated_discussions_output"
ANNOTATION_DIR = "data/annotations_output" 

In [3]:
conv_df = preprocessing.import_conversations(CONVERSATION_DIR)
conv_df.head()

,id,timestamp,users,user_types,moderator,moderator_type,user_prompts,moderator_prompt,ctx_length,conv_variant,user,message
0,132b1278-8811-4145-bdfa-bddcc3ef56f9,24-10-18-19-24,"[WiseOwl71, TechGuru88, GoldenYears66, DarkSha...","[str, str, str, str, str]",moderator,LLMUser,"[You are WiseOwl71 71 years old, Heterosexual,...",You are moderator . Context: You are a human p...,2,vanilla,DarkShadow333,I'm so sick of people making excuses for terro...
0,132b1278-8811-4145-bdfa-bddcc3ef56f9,24-10-18-19-24,"[WiseOwl71, TechGuru88, GoldenYears66, DarkSha...","[str, str, str, str, str]",moderator,LLMUser,"[You are WiseOwl71 71 years old, Heterosexual,...",You are moderator . Context: You are a human p...,2,vanilla,moderator,"@DarkShadow333, I understand your frustration,..."
0,132b1278-8811-4145-bdfa-bddcc3ef56f9,24-10-18-19-24,"[WiseOwl71, TechGuru88, GoldenYears66, DarkSha...","[str, str, str, str, str]",moderator,LLMUser,"[You are WiseOwl71 71 years old, Heterosexual,...",You are moderator . Context: You are a human p...,2,vanilla,TechGuru88,I'm not sure what kind of world we're living i...
0,132b1278-8811-4145-bdfa-bddcc3ef56f9,24-10-18-19-24,"[WiseOwl71, TechGuru88, GoldenYears66, DarkSha...","[str, str, str, str, str]",moderator,LLMUser,"[You are WiseOwl71 71 years old, Heterosexual,...",You are moderator . Context: You are a human p...,2,vanilla,moderator,
0,132b1278-8811-4145-bdfa-bddcc3ef56f9,24-10-18-19-24,"[WiseOwl71, TechGuru88, GoldenYears66, DarkSha...","[str, str, str, str, str]",moderator,LLMUser,"[You are WiseOwl71 71 years old, Heterosexual,...",You are moderator . Context: You are a human p...,2,vanilla,WiseOwl71,I couldn't agree more with @TechGuru88. It's a...


In [4]:
annot_df = preprocessing.import_annotations(ANNOTATION_DIR, round=True, sentinel_value=-1)
annot_df = annot_df[annot_df.toxicity != -1]
annot_df

,conv_id,timestamp,annotator_type,annotator_prompt,ctx_length,message,toxicity
0,e1a8533f-690b-4b4f-a8bc-78ad287f8129,24-11-07-11-57,LLMAnnotator,None,4,"**Comment:** @QueenOfChaos, you're absolutely...",4
0,e1a8533f-690b-4b4f-a8bc-78ad287f8129,24-11-07-11-57,LLMAnnotator,None,4,"**Comment:** @GoldenYears66, you're absolutel...",4
0,e1a8533f-690b-4b4f-a8bc-78ad287f8129,24-11-07-11-57,LLMAnnotator,None,4,"**Comment:** @GoldenYears66, I think your arg...",4
0,e1a8533f-690b-4b4f-a8bc-78ad287f8129,24-11-07-11-57,LLMAnnotator,None,4,"**Comment:** @TechGuru88, I understand your p...",1
0,e1a8533f-690b-4b4f-a8bc-78ad287f8129,24-11-07-11-57,LLMAnnotator,None,4,**Comment:** I'm not sure what's more disturb...,4
...,...,...,...,...,...,...,...
0,55a46846-74f8-463a-9dff-f116bb9daa8c,24-11-07-11-56,LLMAnnotator,None,4,@DarkShadow333 I couldn't agree more with your...,1
0,55a46846-74f8-463a-9dff-f116bb9daa8c,24-11-07-11-56,LLMAnnotator,None,4,@GoldenYears66 I couldn't agree more with your...,1
0,55a46846-74f8-463a-9dff-f116bb9daa8c,24-11-07-11-56,LLMAnnotator,None,4,@TechGuru88 You're so naive. You think the wor...,3
0,55a46846-74f8-463a-9dff-f116bb9daa8c,24-11-07-11-56,LLMAnnotator,None,4,I'm not sure what kind of world @QueenOfChaos ...,2


In [5]:
combined_df = conv_df.merge(annot_df, left_on="id", right_on="conv_id")
combined_df = combined_df[["id", "conv_variant", "user", "message_x", "toxicity"]]
combined_df = combined_df.rename({"message_x": "message"}, axis=1)
combined_df = combined_df.drop_duplicates()
combined_df

,id,conv_variant,user,message,toxicity
0,132b1278-8811-4145-bdfa-bddcc3ef56f9,vanilla,DarkShadow333,I'm so sick of people making excuses for terro...,4
3,132b1278-8811-4145-bdfa-bddcc3ef56f9,vanilla,DarkShadow333,I'm so sick of people making excuses for terro...,1
12,132b1278-8811-4145-bdfa-bddcc3ef56f9,vanilla,DarkShadow333,I'm so sick of people making excuses for terro...,3
21,132b1278-8811-4145-bdfa-bddcc3ef56f9,vanilla,DarkShadow333,I'm so sick of people making excuses for terro...,2
33,132b1278-8811-4145-bdfa-bddcc3ef56f9,vanilla,moderator,"@DarkShadow333, I understand your frustration,...",4
...,...,...,...,...,...
42963,71323709-ef1f-46d1-845e-425f38d44a7c,arguments,moderator,,4
42984,71323709-ef1f-46d1-845e-425f38d44a7c,arguments,doing_my_best24,"I understand your frustration @WiseOwl71, but ...",2
42985,71323709-ef1f-46d1-845e-425f38d44a7c,arguments,doing_my_best24,"I understand your frustration @WiseOwl71, but ...",4
43006,71323709-ef1f-46d1-845e-425f38d44a7c,arguments,moderator,I understand your frustration @doing_my_best24...,2
